In [33]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [34]:
def entropy(y):
    counts = np.bincount(y)
    probabilities = counts[np.nonzero(counts)] / len(y)
    return -np.sum(probabilities * np.log2(probabilities))

In [35]:
def information_gain(X_column, y, split_threshold):
    parent_entropy = entropy(y)
    left_idxs = np.where(X_column <= split_threshold)[0]
    right_idxs = np.where(X_column > split_threshold)[0]
    if len(left_idxs) == 0 or len(right_idxs) == 0:
        return 0
    n = len(y)
    n_left, n_right = len(left_idxs), len(right_idxs)
    e_left, e_right = entropy(y[left_idxs]), entropy(y[right_idxs])
    child_entropy = (n_left / n) * e_left + (n_right / n) * e_right
    return parent_entropy - child_entropy

In [36]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        return self.value is not None

In [37]:
class DecisionTree:
    def __init__(self, max_depth=100, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        self.n_classes = len(set(y))
        self.n_features = X.shape[1]
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(set(y))
        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        best_gain = -1
        split_idx, split_threshold = None, None
        for feature_idx in range(num_features):
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            for t in thresholds:
                gain = information_gain(X_column, y, t)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_threshold = t
        if best_gain == 0:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        left_idxs = np.where(X[:, split_idx] <= split_threshold)[0]
        right_idxs = np.where(X[:, split_idx] > split_threshold)[0]
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(split_idx, split_threshold, left, right)

    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [38]:
def main():
  data = load_iris()
  X, y = data.data, data.target
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  clf = DecisionTree(max_depth=5)
  clf.fit(X_train, y_train)
  preds = clf.predict(X_test)
  acc = np.sum(preds == y_test) / len(y_test)
  print(f"Accuracy: {acc}")

In [39]:
main()

Accuracy: 1.0
